<a href="https://colab.research.google.com/github/syahmishamz/Data-Mining/blob/main/Assignment2/AssignmentPart2_Codes_Section04_0371440.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Mining (ITS61504) Assignment 2**


---






**Seting up display**

In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

#1) Import libraries

**Installation**

In [2]:
!pip install memory_profiler

**Libraries**

In [3]:
import numpy as np
import pandas as pd
import time
import tracemalloc

from memory_profiler import profile
from mlxtend.frequent_patterns import apriori, fpgrowth
from mlxtend.frequent_patterns import association_rules

#2) Clone GitHub Repo

In [4]:
!git clone https://github.com/syahmishamz/Data-Mining.git

Cloning into 'Data-Mining'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 32 (delta 8), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (32/32), 741.50 KiB | 5.79 MiB/s, done.
Resolving deltas: 100% (8/8), done.


#3) Load the Dataset

In [5]:
file = '/content/Data-Mining/Accidental_Drug_Related_Deaths.csv'

In [6]:
data = pd.read_csv(file)
data.head(10)

,Date,Date Type,Age,Sex,Race,Ethnicity,Residence City,Residence County,Residence State,Injury City,...,Xylazine,Gabapentin,Opiate NOS,Heroin/Morph/Codeine,Other Opioid,Any Opioid,Other,ResidenceCityGeo,InjuryCityGeo,DeathCityGeo
0,05/29/2012,Date of death,37.0,Male,Black,NaN,STAMFORD,FAIRFIELD,NaN,STAMFORD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"STAMFORD, CT\n(41.051924, -73.539475)","STAMFORD, CT\n(41.051924, -73.539475)","CT\n(41.575155, -72.738288)"
1,06/27/2012,Date of death,37.0,Male,White,NaN,NORWICH,NEW LONDON,NaN,NORWICH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"NORWICH, CT\n(41.524304, -72.075821)","NORWICH, CT\n(41.524304, -72.075821)","Norwich, CT\n(41.524304, -72.075821)"
2,03/24/2014,Date of death,28.0,Male,White,NaN,HEBRON,NaN,NaN,HEBRON,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"HEBRON, CT\n(41.658069, -72.366324)","HEBRON, CT\n(41.658069, -72.366324)","Marlborough, CT\n(41.632043, -72.461309)"
3,12/31/2014,Date of death,26.0,Female,White,NaN,BALTIC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"BALTIC, CT\n(41.617221, -72.085031)","CT\n(41.575155, -72.738288)","Baltic, CT\n(41.617221, -72.085031)"
4,01/16/2016,Date of death,41.0,Male,White,NaN,SHELTON,FAIRFIELD,CT,SHELTON,...,NaN,NaN,NaN,NaN,NaN,Y,NaN,"SHELTON, CT\n(41.316843, -73.092968)","SHELTON, CT\n(41.316843, -73.092968)","Bridgeport, CT\n(41.179195, -73.189476)"
5,06/13/2017,Date reported,57.0,Male,White,NaN,BLANDFORD,HAMPDEN,MA,ENFIELD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ENFIELD, CT\n(41.976501, -72.591985)","Enfield, CT\n(41.976501, -72.591985)"
6,10/20/2015,Date reported,26.0,Male,White,NaN,DANBURY,FAIRFIELD,CT,DANBURY,...,NaN,NaN,NaN,NaN,NaN,Y,NaN,"DANBURY, CT\n(41.393666, -73.451539)","DANBURY, CT\n(41.393666, -73.451539)","Danbury, CT\n(41.393666, -73.451539)"
7,02/02/2017,Date reported,64.0,Male,White,NaN,MILFORD,NEW HAVEN,CT,MILFORD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MILFORD, CT\n(41.224276, -73.057564)","MILFORD, CT\n(41.224276, -73.057564)","Milford, CT\n(41.224276, -73.057564)"
8,07/03/2018,Date of death,33.0,Male,NaN,NaN,NaN,NaN,NaN,HARTFORD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"CT\n(41.575155, -72.738288)","HARTFORD, CT\n(41.765775, -72.673356)","Hartford, CT\n(41.765775, -72.673356)"
9,05/08/2013,Date of death,23.0,Male,White,NaN,BETHEL,FAIRFIELD,NaN,BETHEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"BETHEL, CT\n(41.371727, -73.407448)","BETHEL, CT\n(41.371727, -73.407448)","Bethel, CT\n(41.371727, -73.407448)"


In [7]:
data.columns #Check variable names

Index(['Date', 'Date Type', 'Age', 'Sex', 'Race', 'Ethnicity',
       'Residence City', 'Residence County', 'Residence State', 'Injury City',
       'Injury County', 'Injury State', 'Injury Place',
       'Description of Injury', 'Death City', 'Death County', 'Death State',
       'Location', 'Location if Other', 'Cause of Death', 'Manner of Death',
       'Other Significant Conditions ', 'Heroin',
       'Heroin death certificate (DC)', 'Cocaine', 'Fentanyl',
       'Fentanyl Analogue', 'Oxycodone', 'Oxymorphone', 'Ethanol',
       'Hydrocodone', 'Benzodiazepine', 'Methadone', 'Meth/Amphetamine',
       'Amphet', 'Tramad', 'Hydromorphone', 'Morphine (Not Heroin)',
       'Xylazine', 'Gabapentin', 'Opiate NOS', 'Heroin/Morph/Codeine',
       'Other Opioid', 'Any Opioid', 'Other', 'ResidenceCityGeo',
       'InjuryCityGeo', 'DeathCityGeo'],
      dtype='object')

In [8]:
data.shape #Check amount of records & variables

(11981, 48)

#4) Data Cleaning

**Select drug variables for association analysis**

In [9]:
drug_columns = [
    'Heroin', 'Cocaine', 'Fentanyl', 'Oxycodone', 'Oxymorphone', 'Ethanol',
    'Hydrocodone', 'Benzodiazepine', 'Methadone', 'Meth/Amphetamine',
    'Amphet', 'Tramad', 'Hydromorphone', 'Morphine (Not Heroin)',
    'Xylazine', 'Gabapentin', 'Opiate NOS', 'Heroin/Morph/Codeine',
    'Other Opioid', 'Any Opioid'
    ]

drug_df = data[drug_columns].notna().astype(bool) #Convert into boolean (T/F)

In [10]:
drug_df.head(10)

,Heroin,Cocaine,Fentanyl,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Meth/Amphetamine,Amphet,Tramad,Hydromorphone,Morphine (Not Heroin),Xylazine,Gabapentin,Opiate NOS,Heroin/Morph/Codeine,Other Opioid,Any Opioid
0,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
5,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
7,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
8,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
9,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


**Support Count for each drug**

In [11]:
print(drug_df.sum())

Heroin                   3578
Cocaine                  4578
Fentanyl                 8049
Oxycodone                1016
Oxymorphone               162
Ethanol                  3201
Hydrocodone               169
Benzodiazepine           2717
Methadone                1078
Meth/Amphetamine          127
Amphet                    431
Tramad                    302
Hydromorphone              77
Morphine (Not Heroin)      59
Xylazine                 1078
Gabapentin                469
Opiate NOS                127
Heroin/Morph/Codeine     2202
Other Opioid              222
Any Opioid               8947
dtype: int64


#5) Apriori Algorithm

*Measuring Runtime and Memory Usage (Before)*

In [12]:
start_time_apriori = time.time()
tracemalloc.start()

**i) Generate frequent itemsets**

In [13]:
#Include only itemsets appearing in at least 10% of transactions
frequent_itemsets_apriori = apriori(drug_df, min_support=0.1, use_colnames=True)
#Convert support in percentage with 3 d.p.
frequent_itemsets_apriori['support_percentage'] = frequent_itemsets_apriori['support'].apply(lambda x: f'{x:.3%}')
#Sort itemsets in descending order
frequent_itemsets_apriori = frequent_itemsets_apriori.sort_values(by=['support'], ascending=False)

**ii) Generate association rules (based on Confidence)**

*Confidence measures how often a rule is true, in this case at least 60% (min_threshold=0.6)*

In [14]:
rules_apriori = association_rules(frequent_itemsets_apriori, metric="confidence", min_threshold=0.6)

*Measuring Runtime and Memory Usage (After)*

In [15]:
end_time_apriori = time.time()
runtime_apriori = end_time_apriori - start_time_apriori
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
memory_usage_apriori = peak / 10**6  #Convert to MB

#6) FP-Growth Algorithm

*Measuring Runtime and Memory Usage (Before)*

In [16]:
start_time_fp = time.time()
tracemalloc.start()

**i) Generate frequent itemsets**

In [17]:
#Include only itemsets appearing in at least 10% of transactions
frequent_itemsets_fp = fpgrowth(drug_df, min_support=0.1, use_colnames=True)
#Convert support in percentage with 3 d.p.
frequent_itemsets_fp['support_percentage'] = frequent_itemsets_fp['support'].apply(lambda x: f'{x:.3%}')
#Sort itemsets in descending order
frequent_itemsets_fp = frequent_itemsets_fp.sort_values(by=['support'], ascending=False)

**ii) Generate association rules**

*Confidence measures how often a rule is true, in this case at least 60% (min_threshold=0.6)*

In [18]:
rules_fp = association_rules(frequent_itemsets_fp, metric="confidence", min_threshold=0.6)

*Measuring Runtime and Memory Usage (After)*

In [19]:
end_time_fp = time.time()
runtime_fp = end_time_fp - start_time_fp
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
memory_usage_fp = peak / 10**6  #Convert to MB

#7) Frequent Itemsets

**i) Apriori**

In [20]:
#Filter itemsets with more than 1 item
filtered_itemsets_ap = frequent_itemsets_apriori[frequent_itemsets_apriori['itemsets'].apply(lambda x: len(x) > 1)]
print("\nFrequent Itemsets by Support (%) using Apriori:")
print(filtered_itemsets_ap[['support_percentage', 'itemsets']])


Frequent Itemsets by Support (%) using Apriori:
   support_percentage                                           itemsets
15            60.429%                             (Any Opioid, Fentanyl)
11            29.221%                              (Any Opioid, Cocaine)
10            27.727%                                (Fentanyl, Cocaine)
22            25.616%                    (Any Opioid, Fentanyl, Cocaine)
16            20.290%                              (Ethanol, Any Opioid)
9             19.556%                               (Any Opioid, Heroin)
18            18.321%                 (Heroin/Morph/Codeine, Any Opioid)
12            18.154%                                (Ethanol, Fentanyl)
8             17.386%                     (Heroin/Morph/Codeine, Heroin)
21            17.361%         (Heroin/Morph/Codeine, Any Opioid, Heroin)
7             16.860%                                 (Heroin, Fentanyl)
17            16.509%                       (Any Opioid, Benzodiazepine)
23

**ii) FP-Growth**

In [21]:
#Filter itemsets with more than 1 item
filtered_itemsets_fp = frequent_itemsets_fp[frequent_itemsets_fp['itemsets'].apply(lambda x: len(x) > 1)]
print("\nFrequent Itemsets by Support (%) using FP-Growth:")
print(filtered_itemsets_fp[['support_percentage', 'itemsets']])


Frequent Itemsets by Support (%) using FP-Growth:
   support_percentage                                           itemsets
13            60.429%                             (Any Opioid, Fentanyl)
7             29.221%                              (Any Opioid, Cocaine)
8             27.727%                                (Fentanyl, Cocaine)
9             25.616%                    (Any Opioid, Fentanyl, Cocaine)
21            20.290%                              (Ethanol, Any Opioid)
10            19.556%                               (Any Opioid, Heroin)
14            18.321%                 (Heroin/Morph/Codeine, Any Opioid)
22            18.154%                                (Ethanol, Fentanyl)
15            17.386%                     (Heroin/Morph/Codeine, Heroin)
17            17.361%         (Heroin/Morph/Codeine, Any Opioid, Heroin)
11            16.860%                                 (Heroin, Fentanyl)
24            16.509%                       (Any Opioid, Benzodiazepine)


#8) Association Rules

**i) Apriori**

In [22]:
print("\nAssociation Rules (Apriori):")

#Include only these rules
rules_apriori_formatted = rules_apriori[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

#Define rules to convert to percentage
apriori_percent_cols = ['support', 'confidence']

#Filter rules for confidence > 80% & lift > 2
rules_apriori_formatted = rules_apriori_formatted[
    (rules_apriori_formatted['confidence'] > 0.8) & (rules_apriori_formatted['lift'] > 2)]

#Convert apriori_percent_cols into percentage with 3 d.p.
for col in apriori_percent_cols:
    rules_apriori_formatted[col] = rules_apriori_formatted[col].map(lambda x: f'{x:.3%}')

#Sort rules by confidence, then lift
rules_apriori_formatted = rules_apriori_formatted.sort_values(by=['confidence', 'lift'], ascending=[False, False])

print(rules_apriori_formatted)


Association Rules (Apriori):
                                     antecedents             consequents  \
31  (Heroin/Morph/Codeine, Any Opioid, Fentanyl)                (Heroin)   
27              (Heroin/Morph/Codeine, Fentanyl)                (Heroin)   
36              (Heroin/Morph/Codeine, Fentanyl)    (Any Opioid, Heroin)   
12            (Heroin/Morph/Codeine, Any Opioid)                (Heroin)   
11                        (Heroin/Morph/Codeine)                (Heroin)   
15                        (Heroin/Morph/Codeine)    (Any Opioid, Heroin)   
14                          (Any Opioid, Heroin)  (Heroin/Morph/Codeine)   
33                (Any Opioid, Heroin, Fentanyl)  (Heroin/Morph/Codeine)   

    support confidence      lift  
31  11.635%    98.516%  3.298823  
27  11.643%    98.309%  3.291884  
36  11.635%    98.238%  5.023439  
12  17.361%    94.761%  3.173084  
11  17.386%    94.596%  3.167559  
15  17.361%    94.460%  4.830219  
14  17.361%    88.775%  4.830219  
33  1

**ii) FP-Growth**

In [23]:
print("\nAssociation Rules (FP-Growth):")

#Include only these rules
rules_fp_formatted = rules_fp[['antecedents','consequents', 'support', 'confidence', 'lift']]

#Define rules to convert to percentage
fp_percent_cols = ['support', 'confidence']

#Filter rules for confidence > 80% & lift > 2
rules_fp_formatted = rules_fp_formatted[
    (rules_fp_formatted['confidence'] > 0.8) & (rules_fp_formatted['lift'] > 2)]

#Convert fp_percent_cols into percentage with 3 d.p.
for col in fp_percent_cols:
    rules_fp_formatted[col] = rules_fp_formatted[col].map(lambda x: f'{x:.3%}')

#Sort rules by confidence, then lift
rules_fp_formatted = rules_fp_formatted.sort_values(by=['confidence', 'lift'], ascending=[False,False])

print(rules_fp_formatted)


Association Rules (FP-Growth):
                                     antecedents             consequents  \
31  (Heroin/Morph/Codeine, Any Opioid, Fentanyl)                (Heroin)   
27              (Heroin/Morph/Codeine, Fentanyl)                (Heroin)   
36              (Heroin/Morph/Codeine, Fentanyl)    (Any Opioid, Heroin)   
12            (Heroin/Morph/Codeine, Any Opioid)                (Heroin)   
11                        (Heroin/Morph/Codeine)                (Heroin)   
15                        (Heroin/Morph/Codeine)    (Any Opioid, Heroin)   
14                          (Any Opioid, Heroin)  (Heroin/Morph/Codeine)   
33                (Any Opioid, Heroin, Fentanyl)  (Heroin/Morph/Codeine)   

    support confidence      lift  
31  11.635%    98.516%  3.298823  
27  11.643%    98.309%  3.291884  
36  11.635%    98.238%  5.023439  
12  17.361%    94.761%  3.173084  
11  17.386%    94.596%  3.167559  
15  17.361%    94.460%  4.830219  
14  17.361%    88.775%  4.830219  
33 

#10) Runtime & Memory Usage

In [24]:
print("\nApriori Algorithm:")
print(f"Runtime: {runtime_apriori:.3f} seconds")
print(f"Memory Usage: {memory_usage_apriori:.3f} MB")

print("\nFP-Growth Algorithm:")
print(f"Runtime: {runtime_fp:.3f} seconds")
print(f"Memory Usage: {memory_usage_fp:.3f} MB")


Apriori Algorithm:
Runtime: 0.176 seconds
Memory Usage: 1.109 MB

FP-Growth Algorithm:
Runtime: 16.836 seconds
Memory Usage: 4.415 MB
